In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import pandas as pd; print('pandas', pd.__version__)
import cv2; print('opencv2', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
numpy 1.16.4
pandas 0.24.2
opencv2 4.1.0
sklearn 0.21.2
tensorflow 1.14.0
keras 2.2.4-tf


# 1. Load Meta File and Annotation

In [2]:
df_meta_train = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TRAIN_META_FILE, index_col=['seriesuid'])
df_meta_train.index = df_meta_train.index.astype('str')
df_meta_test = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_TEST_META_FILE, index_col=['seriesuid'])
df_meta_test.index = df_meta_test.index.astype('str')

print('meta_train:', df_meta_train.shape, '\n', 'meta_test:', df_meta_test.shape)

meta_train: (1470, 11) 
 meta_test: (222, 11)


In [3]:
df_annotation = pd.read_csv(filepath_or_buffer=settings.PREPROCESS_ANNOTATION_FILE, index_col=['seriesuid'])
df_annotation.index = df_annotation.index.astype('str')
df_annotation['label'] = df_annotation['label'].astype('int')
print('annotation:', df_annotation.shape, 'distinct lung:', len(set(df_annotation.index)))

annotation: (12218, 16) distinct lung: 1436


In [4]:
df_annotation.sample(10)

,width,height,slice,vcoordX,vcoordY,vcoordZ,diameterX,diameterY,diameterZ,originX,originY,originZ,spacingX,spacingY,spacingZ,label
seriesuid,,,,,,,,,,,,,,,,
688366,350,350,300,135.009766,193.115234,275.0,8.203125,16.406250,5.000000,-197.899658,-174.658203,-445.534302,0.683594,0.683594,5.0,5
401934,360,360,265,311.835938,177.539062,33.0,30.937500,18.281250,20.000002,-174.000000,-180.000000,-760.400024,0.703125,0.703125,5.0,5
492982,420,420,425,211.640502,186.620985,40.0,7.382808,13.124992,5.000000,-210.000000,-210.000000,-398.000000,0.820312,0.820312,5.0,31
648146,350,350,340,239.257813,101.171875,103.0,12.988281,10.253906,10.000000,-187.731537,-174.658203,-379.792603,0.683594,0.683594,5.0,31
697424,350,350,300,219.091797,157.226562,180.0,12.304688,12.988281,5.000000,-180.468567,-174.658203,-968.334717,0.683594,0.683594,5.0,32
702551,382,382,282,314.478516,213.755859,117.0,4.476562,5.968750,3.000000,-189.854004,-42.799999,-345.670013,0.746094,0.746094,3.0,1
680347,315,315,335,89.516602,211.333008,55.0,4.921875,6.152344,5.000000,-157.192383,-157.192383,371.470825,0.615234,0.615234,5.0,1
516425,311,311,275,160.966797,205.916016,75.0,13.970703,9.111328,5.000000,-153.196289,-316.196289,-510.100006,0.607422,0.607422,5.0,31
689252,392,392,365,219.351562,199.062500,190.0,9.187500,11.484375,5.000000,-191.259399,-195.617188,-508.065796,0.765625,0.765625,5.0,31


# 2. Load Center Coords by Candidates

In [5]:
lungs = list(set(df_meta_train.index))
print('distinct lungs:', len(lungs))

distinct lungs: 1470


In [6]:
df_tasks = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train/tasks_medi.csv', index_col=['seriesuid'])
df_tasks.index = df_tasks.index.astype('str')

print('tasks:', df_tasks.shape, 'distinct lung:', len(set(df_tasks.index)))

tasks: (25558, 7) distinct lung: 100


# 3. Predict Results by Task_test.csv

In [7]:
df_tasks_nms = df_tasks

# df_tasks_nms = pd.DataFrame(list_tasks_nms, columns=['seriesuid','vcoordX','vcoordY','vcoordZ'])
# df_tasks_nms.index = df_tasks_nms.index.astype('str')

print('tasks:', df_tasks_nms.shape, 'distinct lung:', len(set(df_tasks_nms.index)))

tasks: (25558, 7) distinct lung: 100


In [8]:
df_tasks_nms.head(10)

,vcoordX,vcoordY,vcoordZ,diametersX,diametersY,diametersZ,probability
seriesuid,,,,,,,
655457,163,41,26,10,9,7,0.995914
655457,71,63,38,34,30,17,0.999296
655457,234,63,32,11,13,12,0.998383
655457,41,95,31,17,26,20,0.998630
655457,27,137,27,3,8,5,0.969941
655457,215,121,33,14,8,8,0.997508
655457,282,127,32,14,34,27,0.999036
655457,32,164,42,28,44,43,0.998468
655457,196,155,36,12,12,9,0.999245


In [9]:
import tensorflow.keras.backend as K
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [10]:
from tensorflow.keras.models import load_model
output_dir = './output/densenet3d/1564559557/' # labels in center coord
model = load_model(output_dir + "model-best.hd5")

W0731 23:53:23.098431 140082809853760 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 23:53:23.099493 140082809853760 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0731 23:53:23.106259 140082809853760 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprec

In [11]:
labels = [0, 31, 32]
batch_size = 64
threshold_probability = 0.7
wtype = 'medi'
plot = False

In [12]:
def plot_cube(cube, rows=8, cols=8):
    num_z, h, w = cube.shape # z,y,x
    assert rows * cols == num_z
    
    img = np.zeros((rows * h, cols * w), dtype=np.uint8)

    for row in range(rows):
        for col in range(cols):
            y = row * h
            x = col * w
            img[y:(y+h), x:(x+w)] = cube[row * cols + col,:,:]
    
    fig, axs = plt.subplots(1, 1, figsize=(32, 32), sharex=True, sharey=True)
    axs.imshow(img, cmap='gray')
    plt.show()
    

In [13]:
list_results = []
print(wtype, 'window')
count_lung = 0
for uid in set(df_tasks_nms.index):
    tasks = df_tasks_nms.loc[[uid]] # DataFrame
    meta = df_meta_train.loc[uid] # slice
    
    print('lung:', uid, 'tasks:', len(tasks))
#     print(meta)
                 

    lung_l, mask_l = helper.load_lung_array(uid, int(meta.width), int(meta.height), int(meta.slice), 
                                            mode='train', wtype=wtype)

    lung = (lung_l*(mask_l>0)).astype('float') / 255

    # convert coordX to vcoordX
#     tasks['vcoordX'] = tasks['coordX'] - meta.originX
#     tasks['vcoordY'] = tasks['coordY'] - meta.originY
#     tasks['vcoordZ'] = tasks['coordZ'] - meta.originZ

    count = 0
    for idx, item in tasks.iterrows():
        if count % (batch_size*100) == 0:
            X_item = []
            X_test = []
        
        flag_range = True
        if (item.vcoordX < 100) or (item.vcoordX > 250):
            flag_range = False
        if (item.vcoordY < 50) or (item.vcoordY > 250):
            flag_range = False
        if item.vcoordZ > 350:
            flag_range = False

        # patch without mask       
        cube = helper.get_cube_from_lung_array(lung, item.vcoordX, item.vcoordY, item.vcoordZ, 
                                               block_size=32)

        if np.sum(cube) > 0 and flag_range:
            X_item.append(item)
            X_test.append(cube)
#         else:
#             print('nothing in coord:', '\n', item)

        if (count % (batch_size*100) == (batch_size*100 - 1) or count == len(tasks) - 1) and len(X_test) > 0:
            assert len(X_item) == len(X_test) or count == len(tasks) - 1

            # Make predictions
            X_test = np.expand_dims(np.asarray(X_test), axis=-1)
            predictions_test = model.predict(X_test, batch_size=batch_size*5, verbose=1)

            for i in range(len(X_item)):
                itm = X_item[i]
                if np.argmax(predictions_test[i]) > 0:
                    category = int(np.argmax(predictions_test[i]))
                    probability = float(np.max(predictions_test[i]))

                    result = {}
                    result['seriesuid'] = uid
                    result['coordX'] = itm.vcoordX + meta.originX
                    result['coordY'] = itm.vcoordY + meta.originY
                    result['coordZ'] = itm.vcoordZ + meta.originZ
                    result['class'] = helper.get_origin_class_by_predicted_value(category, labels)
                    result['probability'] = probability
                    result['wtype'] = wtype

                    if probability > threshold_probability:
                        print(result)
                        if plot:
                            plot_cube((cube*255).astype(np.uint8), rows=4, cols=8)
                        list_results.append(result)

        count += 1
   
    # save the csv by per lung
    df_results = pd.DataFrame(list_results, columns=['seriesuid','coordX','coordY','coordZ','class','probability','wtype'])
    df_results = df_results.set_index('seriesuid')
    df_results.index = df_results.index.astype('str')
    
    count_lung += 1

    print('total:', df_results.shape, 'lung:', len(set(df_results.index)), '/', count_lung)

    df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype +'.csv', encoding='utf-8')

    print('-'*100)
    

medi window
lung: 656778 tasks: 144
46/46 [==============================] - 3s 71ms/sample
{'seriesuid': '656778', 'coordX': 12.8251953125, 'coordY': -120.1748046875, 'coordZ': -253.20001220703125, 'class': 32, 'probability': 0.9157347083091736, 'wtype': 'medi'}
{'seriesuid': '656778', 'coordX': 22.8251953125, 'coordY': -105.1748046875, 'coordZ': -290.20001220703125, 'class': 32, 'probability': 0.7596288919448853, 'wtype': 'medi'}
{'seriesuid': '656778', 'coordX': 52.8251953125, 'coordY': -74.1748046875, 'coordZ': -174.20001220703125, 'class': 31, 'probability': 0.8463409543037415, 'wtype': 'medi'}
total: (3, 6) lung: 1 / 1
----------------------------------------------------------------------------------------------------
lung: 388509 tasks: 301
94/94 [==============================] - 0s 3ms/sample
{'seriesuid': '388509', 'coordX': -97.0, 'coordY': -34.0, 'coordZ': -248.0, 'class': 31, 'probability': 0.9859522581100464, 'wtype': 'medi'}
{'seriesuid': '388509', 'coordX': -27.0, 'coor

111/111 [==============================] - 0s 3ms/sample
{'seriesuid': '414510', 'coordX': 58.096405029296875, 'coordY': -94.640625, 'coordZ': -369.6014404296875, 'class': 32, 'probability': 0.7232553958892822, 'wtype': 'medi'}
{'seriesuid': '414510', 'coordX': 2.096405029296875, 'coordY': -26.640625, 'coordZ': -385.6014404296875, 'class': 31, 'probability': 0.9944610595703125, 'wtype': 'medi'}
{'seriesuid': '414510', 'coordX': -0.903594970703125, 'coordY': 50.359375, 'coordZ': -383.6014404296875, 'class': 32, 'probability': 0.9974032044410706, 'wtype': 'medi'}
{'seriesuid': '414510', 'coordX': -59.903594970703125, 'coordY': -65.640625, 'coordZ': -315.6014404296875, 'class': 31, 'probability': 0.8331588506698608, 'wtype': 'medi'}
{'seriesuid': '414510', 'coordX': -11.903594970703125, 'coordY': -44.640625, 'coordZ': -300.6014404296875, 'class': 31, 'probability': 0.999994158744812, 'wtype': 'medi'}
{'seriesuid': '414510', 'coordX': -65.90359497070312, 'coordY': -16.640625, 'coordZ': -31

35/35 [==============================] - 0s 3ms/sample
{'seriesuid': '656238', 'coordX': -7.3520050048828125, 'coordY': -23.110305786132812, 'coordZ': -732.0, 'class': 31, 'probability': 0.9979829788208008, 'wtype': 'medi'}
{'seriesuid': '656238', 'coordX': -55.35200500488281, 'coordY': -26.110305786132812, 'coordZ': -944.0, 'class': 31, 'probability': 0.8522987365722656, 'wtype': 'medi'}
{'seriesuid': '656238', 'coordX': -8.352005004882812, 'coordY': -23.110305786132812, 'coordZ': -731.0, 'class': 31, 'probability': 0.9999822378158569, 'wtype': 'medi'}
{'seriesuid': '656238', 'coordX': -20.352005004882812, 'coordY': -38.11030578613281, 'coordZ': -727.0, 'class': 32, 'probability': 0.9487025141716003, 'wtype': 'medi'}
total: (101, 6) lung: 10 / 10
----------------------------------------------------------------------------------------------------
lung: 664003 tasks: 226
75/75 [==============================] - 0s 3ms/sample
{'seriesuid': '664003', 'coordX': -26.677734375, 'coordY': -18

90/90 [==============================] - 0s 3ms/sample
{'seriesuid': '688185', 'coordX': 51.956695556640625, 'coordY': -61.685546875, 'coordZ': -254.9302978515625, 'class': 32, 'probability': 0.9396719932556152, 'wtype': 'medi'}
{'seriesuid': '688185', 'coordX': -4.043304443359375, 'coordY': -0.685546875, 'coordZ': -251.9302978515625, 'class': 31, 'probability': 0.9724534153938293, 'wtype': 'medi'}
{'seriesuid': '688185', 'coordX': 59.956695556640625, 'coordY': 35.314453125, 'coordZ': -212.9302978515625, 'class': 32, 'probability': 0.9389243125915527, 'wtype': 'medi'}
{'seriesuid': '688185', 'coordX': 50.956695556640625, 'coordY': -29.685546875, 'coordZ': -196.9302978515625, 'class': 31, 'probability': 0.9996641874313354, 'wtype': 'medi'}
{'seriesuid': '688185', 'coordX': -41.043304443359375, 'coordY': -3.685546875, 'coordZ': -181.9302978515625, 'class': 32, 'probability': 0.9281505346298218, 'wtype': 'medi'}
{'seriesuid': '688185', 'coordX': -31.043304443359375, 'coordY': 34.314453125

57/57 [==============================] - 0s 3ms/sample
{'seriesuid': '655376', 'coordX': 13.975997924804688, 'coordY': 93.0, 'coordZ': -363.9599914550781, 'class': 31, 'probability': 0.9769126176834106, 'wtype': 'medi'}
{'seriesuid': '655376', 'coordX': -106.02400207519531, 'coordY': 88.0, 'coordZ': -259.9599914550781, 'class': 32, 'probability': 0.9999992847442627, 'wtype': 'medi'}
{'seriesuid': '655376', 'coordX': -90.02400207519531, 'coordY': 79.0, 'coordZ': -244.95999145507812, 'class': 32, 'probability': 0.9998939037322998, 'wtype': 'medi'}
{'seriesuid': '655376', 'coordX': 33.97599792480469, 'coordY': 128.0, 'coordZ': -224.95999145507812, 'class': 31, 'probability': 0.9999935626983643, 'wtype': 'medi'}
{'seriesuid': '655376', 'coordX': 5.9759979248046875, 'coordY': 155.0, 'coordZ': -351.9599914550781, 'class': 31, 'probability': 0.9054905772209167, 'wtype': 'medi'}
{'seriesuid': '655376', 'coordX': -105.02400207519531, 'coordY': 84.0, 'coordZ': -261.9599914550781, 'class': 32, 'p

...

41/41 [==============================] - 0s 4ms/sample
{'seriesuid': '674849', 'coordX': 39.8623046875, 'coordY': -164.1376953125, 'coordZ': -114.5, 'class': 31, 'probability': 0.8641319274902344, 'wtype': 'medi'}
{'seriesuid': '674849', 'coordX': 29.8623046875, 'coordY': -145.1376953125, 'coordZ': -130.5, 'class': 31, 'probability': 0.9968187808990479, 'wtype': 'medi'}
{'seriesuid': '674849', 'coordX': 30.8623046875, 'coordY': -139.1376953125, 'coordZ': -48.5, 'class': 31, 'probability': 0.9999573230743408, 'wtype': 'medi'}
{'seriesuid': '674849', 'coordX': 37.8623046875, 'coordY': -117.1376953125, 'coordZ': -54.5, 'class': 31, 'probability': 0.9994946718215942, 'wtype': 'medi'}
{'seriesuid': '674849', 'coordX': -59.1376953125, 'coordY': -141.1376953125, 'coordZ': -188.5, 'class': 31, 'probability': 0.7190855145454407, 'wtype': 'medi'}
{'seriesuid': '674849', 'coordX': -61.1376953125, 'coordY': -110.1376953125, 'coordZ': -190.5, 'class': 31, 'probability': 0.8544683456420898, 'wtype':

{'seriesuid': '656403', 'coordX': -90.02999877929688, 'coordY': 87.0, 'coordZ': -255.10000610351562, 'class': 32, 'probability': 0.9999735355377197, 'wtype': 'medi'}
{'seriesuid': '656403', 'coordX': -35.029998779296875, 'coordY': 141.0, 'coordZ': -213.10000610351562, 'class': 32, 'probability': 0.7019333839416504, 'wtype': 'medi'}
{'seriesuid': '656403', 'coordX': 0.970001220703125, 'coordY': 171.0, 'coordZ': -150.10000610351562, 'class': 32, 'probability': 0.9945032596588135, 'wtype': 'medi'}
{'seriesuid': '656403', 'coordX': -8.029998779296875, 'coordY': 148.0, 'coordZ': -206.10000610351562, 'class': 32, 'probability': 0.9763309359550476, 'wtype': 'medi'}
total: (795, 6) lung: 82 / 85
----------------------------------------------------------------------------------------------------
lung: 648428 tasks: 160
67/67 [==============================] - 0s 3ms/sample
{'seriesuid': '648428', 'coordX': 28.628250122070284, 'coordY': -80.71875, 'coordZ': 405.9354248046875, 'class': 31, 'proba

121/121 [==============================] - 0s 3ms/sample
{'seriesuid': '639006', 'coordX': 32.988967895507784, 'coordY': 25.2734375, 'coordZ': 480.0509033203125, 'class': 31, 'probability': 0.9916691780090332, 'wtype': 'medi'}
{'seriesuid': '639006', 'coordX': 49.988967895507784, 'coordY': -11.7265625, 'coordZ': 516.0509033203125, 'class': 31, 'probability': 0.9802456498146057, 'wtype': 'medi'}
{'seriesuid': '639006', 'coordX': 93.98896789550778, 'coordY': -83.7265625, 'coordZ': 517.0509033203125, 'class': 31, 'probability': 0.9246212840080261, 'wtype': 'medi'}
{'seriesuid': '639006', 'coordX': -28.011032104492216, 'coordY': 12.2734375, 'coordZ': 501.0509033203125, 'class': 32, 'probability': 0.9790579676628113, 'wtype': 'medi'}
{'seriesuid': '639006', 'coordX': 54.988967895507784, 'coordY': 12.2734375, 'coordZ': 543.0509033203125, 'class': 32, 'probability': 0.9834811091423035, 'wtype': 'medi'}
{'seriesuid': '639006', 'coordX': 6.988967895507784, 'coordY': -9.7265625, 'coordZ': 581.05

98/98 [==============================] - 0s 3ms/sample
{'seriesuid': '655633', 'coordX': 28.294921875, 'coordY': -155.705078125, 'coordZ': -146.29998779296875, 'class': 31, 'probability': 0.9996373653411865, 'wtype': 'medi'}
{'seriesuid': '655633', 'coordX': 24.294921875, 'coordY': -92.705078125, 'coordZ': -118.29998779296875, 'class': 31, 'probability': 0.9142429828643799, 'wtype': 'medi'}
{'seriesuid': '655633', 'coordX': -4.705078125, 'coordY': -194.705078125, 'coordZ': -282.29998779296875, 'class': 32, 'probability': 0.7763150930404663, 'wtype': 'medi'}
{'seriesuid': '655633', 'coordX': 4.294921875, 'coordY': -192.705078125, 'coordZ': -242.29998779296875, 'class': 31, 'probability': 0.9629042148590088, 'wtype': 'medi'}
{'seriesuid': '655633', 'coordX': 18.294921875, 'coordY': -95.705078125, 'coordZ': -181.29998779296875, 'class': 32, 'probability': 0.9432795643806458, 'wtype': 'medi'}
{'seriesuid': '655633', 'coordX': 28.294921875, 'coordY': -155.705078125, 'coordZ': -145.299987792

46/46 [==============================] - 0s 3ms/sample
{'seriesuid': '682499', 'coordX': -36.654296875, 'coordY': -135.654296875, 'coordZ': -469.20001220703125, 'class': 32, 'probability': 0.999991774559021, 'wtype': 'medi'}
{'seriesuid': '682499', 'coordX': -23.654296875, 'coordY': -163.654296875, 'coordZ': -443.20001220703125, 'class': 32, 'probability': 0.9913901090621948, 'wtype': 'medi'}
total: (928, 6) lung: 96 / 100
----------------------------------------------------------------------------------------------------


In [14]:
df_results = pd.read_csv(filepath_or_buffer=settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype +'.csv', index_col=['seriesuid'])
df_results.index = df_results.index.astype('str')

print('results:', df_results.shape, 'distinct lung:', len(set(df_results.index)))

results: (928, 6) distinct lung: 96


In [15]:
df_results = df_results[df_results['probability'] > 0.95]
df_results = df_results.drop(columns=['wtype'])
print(df_results.shape, 'distinct lung:', len(set(df_results.index)))

(627, 5) distinct lung: 89


In [16]:
df_results.to_csv(settings.SUBMISSION_DIR + 'train' + '/submission_' + wtype + '_filtered.csv', encoding='utf-8')

print('31:', len(df_results[df_results['class']==31]), '32:', len(df_results[df_results['class']==32]))
print('1:', len(df_results[df_results['class']==1]), '5:', len(df_results[df_results['class']==5]))

31: 419 32: 208
1: 0 5: 0
